In [1]:
from PIL import Image

In [2]:
image_path = "ryan.jpg"
img = Image.open(image_path)

In [4]:
width, height = img.size

#kuwahara filter time
for y in range(height):
    for x in range(width):
        r, g, b = img.getpixel((x, y))

        img.putpixel((x, y), (r, r, r))
# img.save("ryan_modified.jpg")

In [5]:
filtered_img = Image.new('RGB', (width, height), "black")

In [6]:
import colorsys

In [10]:
def get_mean_and_SD(img, x_min, x_max, y_min, y_max):
    mean = 0
    rgb_mean = [0, 0, 0]
    SD = 0
    for y in range(y_min, y_max):
        for x in range(x_min, x_max):
            #before adding to mean, convert the rgb to hsv and use the v value:
            h, s, v = colorsys.rgb_to_hsv(img.getpixel((x, y))[0], img.getpixel((x, y))[1], img.getpixel((x, y))[2])
            rgb_mean[0] += img.getpixel((x, y))[0]
            rgb_mean[1] += img.getpixel((x, y))[1]
            rgb_mean[2] += img.getpixel((x, y))[2]
            mean += v
    mean //= (x_max - x_min) * (y_max - y_min)
    rgb_mean[0] //= (x_max - x_min) * (y_max - y_min)
    rgb_mean[1] //= (x_max - x_min) * (y_max - y_min)
    rgb_mean[2] //= (x_max - x_min) * (y_max - y_min)
    for y in range(y_min, y_max):
        for x in range(x_min, x_max):
            h, s, v = colorsys.rgb_to_hsv(img.getpixel((x, y))[0], img.getpixel((x, y))[1], img.getpixel((x, y))[2])
            SD += (v - mean) ** 2
    SD /= (x_max - x_min) * (y_max - y_min)
    SD = SD ** 0.5
    return rgb_mean, SD

def kuwahara(img, x, y, square_size):
    #now its a ryan_modified is a grayscale image
    #create a new image to store the filtered image
    #odd sized square
    #iterate through each square
    mean_tl, sd_tl = get_mean_and_SD(img, x - square_size // 2, x, y - square_size // 2, y)
    mean_tr, sd_tr = get_mean_and_SD(img, x + 1, (x + 1) + square_size // 2, y - square_size // 2, y)
    mean_bl, sd_bl = get_mean_and_SD(img, x - square_size // 2, x, y + 1, (y + 1) + square_size // 2)
    mean_br, sd_br = get_mean_and_SD(img, x + 1, (x + 1) + square_size // 2, y + 1, (y + 1) + square_size // 2)
    #choose corresponding mean based on smallest SD
    mean = mean_tl
    if sd_tr <= sd_tl and sd_tr <= sd_bl and sd_tr <= sd_br:
        mean = mean_tr
    elif sd_bl <= sd_tl and sd_bl <= sd_tr and sd_bl <= sd_br:
        mean = mean_bl
    elif sd_br <= sd_tl and sd_br <= sd_tr and sd_br <= sd_bl:
        mean = mean_br
    #set pixel to mean
    filtered_img.putpixel((x, y), (mean[0], mean[1], mean[2]))

In [25]:
#non parallelized version

square_size = 3
passes = 1

#just iterate through x and y coords and call kuwahara on each
img = Image.open("ryan.jpg")

for i in range(passes):
    for y in range(square_size // 2, height - square_size // 2):
        for x in range(square_size // 2, width - square_size // 2):
            kuwahara(img, x, y, square_size)
file_name = "ryan_color_kuwahara" + str(square_size) + "_" + str(passes) + ".jpg"
filtered_img.save(file_name)



In [16]:
import concurrent.futures
import numpy as np

In [28]:
def process_image(img, square_size):
    height, width = img.size[:2]
    results = []

    with concurrent.futures.ThreadPoolExecutor() as executor:
        # Adjust the number of threads based on your requirements
        num_threads = 4

        # Calculate the number of rows each thread should process
        rows_per_thread = (height - square_size) // num_threads

        # Submit tasks to the ThreadPoolExecutor
        futures = []
        for i in range(num_threads):
            start_row = square_size // 2 + i * rows_per_thread
            end_row = start_row + rows_per_thread
            if i == num_threads - 1:
                end_row = height - square_size // 2

            futures.append(
                executor.submit(process_block, img, square_size, start_row, end_row)
            )

        # Wait for all tasks to complete
        concurrent.futures.wait(futures)

        # Retrieve results
    #     results = [future.result() for future in futures]

    # return results

#splittin this by row chunks i guess
def process_block(img, square_size, start_row, end_row):
    height, width = img.size[:2]
    processed_blocks = []

    for y in range(start_row, end_row):
        for x in range(square_size // 2, width - square_size // 2):
            kuwahara(img, x, y, square_size)

    return processed_blocks

square_size = 3
passes = 1

img = Image.open("ryan.jpg")


results = process_image(img, square_size)

file_name = "ryan_color_kuwahara" + str(square_size) + "__" + str(passes) + ".jpg"
filtered_img.save(file_name)